In [293]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tamil-nlp/tamil_movie_reviews_train.csv
/kaggle/input/tamil-nlp/tamil_movie_reviews_test.csv
/kaggle/input/tamil-nlp/tamil_news_train.csv
/kaggle/input/tamil-nlp/tamil_news_test.csv
/kaggle/input/tamil-nlp/tamil_thirukkural_test.csv
/kaggle/input/tamil-nlp/tamil_thirukkural_train.csv


In [329]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [295]:
train = pd.read_csv('/kaggle/input/tamil-nlp/tamil_news_train.csv')
test = pd.read_csv('/kaggle/input/tamil-nlp/tamil_news_train.csv')

In [296]:
train.head()
test.head()
train.shape
test.shape

,NewsInEnglish,NewsInTamil,Category,CategoryInTamil
0,2017/May/19941-miss-iraq-competition-after-43-...,ஈராக்கில் 43 ஆண்டுகள் கழித்து அழகிப்போட்டி,world,உலகம்
1,2018/Dec/55921-viswasam-s-rustic-folk-song-has...,இந்திய அளவில் ட்ரெண்ட் ஆன அஜித்தின் ‘தள்லே தில...,cinema,சினிமா
2,2018/Dec/55929-inspector-who-bought-insurance-...,சொந்த செலவில் வாகன காப்பீடு எடுத்து கொடுத்த கா...,tamilnadu,தமிழ்நாடு
3,2017/Jan/16161-morarji-desai-budget.txt,பிறந்தநாளில் பட்ஜெட் தாக்கல் செய்தவர்,india,இந்தியா
4,2018/Jun/46476-i-have-not-joined-bjp-or-anyoth...,“நானா? பாஜகவில் இணைந்துவிட்டேனா” - வரலட்சுமி வ...,cinema,சினிமா


,NewsInEnglish,NewsInTamil,Category,CategoryInTamil
0,2017/May/19941-miss-iraq-competition-after-43-...,ஈராக்கில் 43 ஆண்டுகள் கழித்து அழகிப்போட்டி,world,உலகம்
1,2018/Dec/55921-viswasam-s-rustic-folk-song-has...,இந்திய அளவில் ட்ரெண்ட் ஆன அஜித்தின் ‘தள்லே தில...,cinema,சினிமா
2,2018/Dec/55929-inspector-who-bought-insurance-...,சொந்த செலவில் வாகன காப்பீடு எடுத்து கொடுத்த கா...,tamilnadu,தமிழ்நாடு
3,2017/Jan/16161-morarji-desai-budget.txt,பிறந்தநாளில் பட்ஜெட் தாக்கல் செய்தவர்,india,இந்தியா
4,2018/Jun/46476-i-have-not-joined-bjp-or-anyoth...,“நானா? பாஜகவில் இணைந்துவிட்டேனா” - வரலட்சுமி வ...,cinema,சினிமா


(14521, 4)

(14521, 4)

In [297]:
train.isnull().any()
test.isnull().any()

NewsInEnglish      False
NewsInTamil        False
Category           False
CategoryInTamil    False
dtype: bool

NewsInEnglish      False
NewsInTamil        False
Category           False
CategoryInTamil    False
dtype: bool

In [298]:
train = train.drop_duplicates().reset_index(drop=True)
test = test.drop_duplicates().reset_index(drop=True)
train.shape
test.shape

(10579, 4)

(10579, 4)

In [299]:
stop_words = stopwords.words('english')

train.NewsInEnglish = train.NewsInEnglish.str.lower()
train.NewsInEnglish = train.NewsInEnglish.str.replace('[^\w\s]',' ')
train.NewsInEnglish = train.NewsInEnglish.str.replace('\d+', ' ')
train.NewsInEnglish = train.NewsInEnglish.str.replace(' txt', '')
train.NewsInEnglish = train.NewsInEnglish.apply(lambda x: [item for item in [x] if item not in stop_words])
count = 0
for i in range(len(train.NewsInEnglish)):
    train.NewsInEnglish[count] = train.NewsInEnglish[count][0].replace('  ', '')
#     train.NewsInEnglish[count] = train.NewsInEnglish[count][0].split(' ')
    count = count + 1
    
train.NewsInTamil = train.NewsInTamil.str.lower()
train.NewsInTamil = train.NewsInTamil.str.replace('[^\w\s]',' ')
train.NewsInTamil = train.NewsInTamil.str.replace('\d+', ' ')

test.NewsInTamil = test.NewsInTamil.str.lower()
test.NewsInTamil = test.NewsInTamil.str.replace('[^\w\s]',' ')
test.NewsInTamil = test.NewsInTamil.str.replace('\d+', ' ')

test.NewsInEnglish = test.NewsInEnglish.str.lower()
test.NewsInEnglish = test.NewsInEnglish.str.replace('[^\w\s]',' ')
test.NewsInEnglish = test.NewsInEnglish.str.replace('\d+', ' ')
test.NewsInEnglish = test.NewsInEnglish.str.replace(' txt', '')
test.NewsInEnglish = test.NewsInEnglish.apply(lambda x: [item for item in [x] if item not in stop_words])    
count = 0
for i in range(len(test.NewsInEnglish)):
    test.NewsInEnglish[count] = test.NewsInEnglish[count][0].replace('  ', '')
#     test.NewsInEnglish[count] = test.NewsInEnglish[count][0].split(' ')
    count = count + 1
    
train.head()
test.head()

,NewsInEnglish,NewsInTamil,Category,CategoryInTamil
0,may miss iraq competition after years,ஈர க க ல ஆண ட கள கழ த த அழக ப ப ட ட,world,உலகம்
1,dec viswasam s rustic folk song has been released,இந த ய அளவ ல ட ர ண ட ஆன அஜ த த ன தள ல த ல...,cinema,சினிமா
2,dec inspector who bought insurance at his own ...,ச ந த ச லவ ல வ கன க ப ப ட எட த த க ட த த க ...,tamilnadu,தமிழ்நாடு
3,jan morarji desai budget,ப றந தந ள ல பட ஜ ட த க கல ச ய தவர,india,இந்தியா
4,jun i have not joined bjp or anyother politica...,ந ன ப ஜகவ ல இண ந த வ ட ட ன வரலட ச ம வ...,cinema,சினிமா


,NewsInEnglish,NewsInTamil,Category,CategoryInTamil
0,may miss iraq competition after years,ஈர க க ல ஆண ட கள கழ த த அழக ப ப ட ட,world,உலகம்
1,dec viswasam s rustic folk song has been released,இந த ய அளவ ல ட ர ண ட ஆன அஜ த த ன தள ல த ல...,cinema,சினிமா
2,dec inspector who bought insurance at his own ...,ச ந த ச லவ ல வ கன க ப ப ட எட த த க ட த த க ...,tamilnadu,தமிழ்நாடு
3,jan morarji desai budget,ப றந தந ள ல பட ஜ ட த க கல ச ய தவர,india,இந்தியா
4,jun i have not joined bjp or anyother politica...,ந ன ப ஜகவ ல இண ந த வ ட ட ன வரலட ச ம வ...,cinema,சினிமா


# English Classification

In [302]:
vect = TfidfVectorizer(ngram_range=(1,1),stop_words='english',min_df=100)
dtm = vect.fit_transform(train.NewsInEnglish)
vect.get_feature_names()
dtm.shape

['actor',
 'actress',
 'apr',
 'arrested',
 'aug',
 'bjp',
 'case',
 'chennai',
 'chief',
 'cm',
 'congress',
 'court',
 'day',
 'dead',
 'death',
 'dec',
 'delhi',
 'dmk',
 'election',
 'feb',
 'film',
 'gaja',
 'gandhi',
 'government',
 'govt',
 'high',
 'india',
 'indian',
 'issue',
 'jan',
 'jul',
 'jun',
 'kamal',
 'kerala',
 'killed',
 'kohli',
 'man',
 'mar',
 'meet',
 'minister',
 'mk',
 'modi',
 'nadu',
 'new',
 'nov',
 'oct',
 'old',
 'palanisamy',
 'people',
 'pm',
 'police',
 'president',
 'protest',
 'rahul',
 'rain',
 'rajinikanth',
 'release',
 'rs',
 'said',
 'says',
 'sep',
 'stalin',
 'suicide',
 'supreme',
 'tamil',
 'tamilnadu',
 'team',
 'test',
 'tn',
 'today',
 'vijay',
 'virat',
 'vs',
 'woman',
 'women',
 'world',
 'year']

(10579, 77)

In [318]:
train_features, test_features, train_labels, test_labels = train_test_split(dtm, train['Category'], test_size=.20)

In [323]:
gnb = GaussianNB()
gnb.fit(train_features.toarray(), train_labels)
nb_pred_train = gnb.predict(train_features.toarray())
nb_pred_test = gnb.predict(test_features.toarray())

GaussianNB(priors=None, var_smoothing=1e-09)

In [331]:
def multiclass_roc_auc_score(truth, pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)
    pred = lb.transform(pred)

    return roc_auc_score(truth, pred, average=average)

In [332]:
print(classification_report(test_labels, nb_pred_test))
print('Naive Bayes baseline: ' + str(multiclass_roc_auc_score(train_labels, nb_pred_train)))
print('Naive Bayes: ' + str(multiclass_roc_auc_score(test_labels, nb_pred_test)))

              precision    recall  f1-score   support

      cinema       0.70      0.29      0.41       328
       india       0.76      0.09      0.17       506
    politics       0.33      0.40      0.36       172
      sports       0.30      0.55      0.38       225
   tamilnadu       0.82      0.17      0.28       716
       world       0.11      0.76      0.19       169

    accuracy                           0.28      2116
   macro avg       0.50      0.38      0.30      2116
weighted avg       0.63      0.28      0.28      2116

Naive Bayes baseline: 0.631497214484437
Naive Bayes: 0.6213193164701862


# Tamil classification

In [333]:
vect = TfidfVectorizer(ngram_range=(1,1), min_df=100)
dtm = vect.fit_transform(train.NewsInTamil)
vect.get_feature_names()
dtm.shape

['ஃப',
 'அட',
 'அண',
 'அத',
 'அன',
 'அப',
 'அம',
 'அர',
 'அரச',
 'அற',
 'ஆச',
 'ஆட',
 'ஆண',
 'ஆய',
 'ஆர',
 'இட',
 'இண',
 'இத',
 'இந',
 'இன',
 'இயக',
 'இர',
 'இல',
 'இள',
 'உச',
 'உத',
 'உய',
 'உயர',
 'உர',
 'உற',
 'உள',
 'எச',
 'எட',
 'எத',
 'என',
 'எம',
 'எல',
 'எஸ',
 'ஏன',
 'ஏற',
 'ஒர',
 'கட',
 'கண',
 'கத',
 'கன',
 'கப',
 'கம',
 'கமல',
 'கர',
 'கல',
 'கள',
 'கழ',
 'கவ',
 'சச',
 'சட',
 'சத',
 'சந',
 'சன',
 'சம',
 'சர',
 'சல',
 'ஜக',
 'ஜய',
 'ஞர',
 'டக',
 'டங',
 'டத',
 'டன',
 'டப',
 'டம',
 'டர',
 'டல',
 'ணவ',
 'ணவர',
 'தகவல',
 'தங',
 'தட',
 'தத',
 'தந',
 'தன',
 'தப',
 'தம',
 'தய',
 'தர',
 'தரவ',
 'தற',
 'தல',
 'தலம',
 'தவ',
 'நட',
 'நர',
 'னகரன',
 'னத',
 'னம',
 'னர',
 'பக',
 'பட',
 'படத',
 'பண',
 'பத',
 'பந',
 'பன',
 'பம',
 'பர',
 'பற',
 'பல',
 'பள',
 'மக',
 'மண',
 'மத',
 'மன',
 'மர',
 'மற',
 'மல',
 'யத',
 'யன',
 'யம',
 'யர',
 'யல',
 'ரச',
 'ரஜ',
 'ரட',
 'ரண',
 'ரத',
 'ரதமர',
 'ரன',
 'ரப',
 'ரம',
 'ரய',
 'ரர',
 'ரல',
 'ரள',
 'ரவ',
 'றத',
 'றந',
 'றப',
 'றம',
 'லட',
 'லம',
 'ளக',
 'ளர',

(10579, 155)

In [335]:
train_features, test_features, train_labels, test_labels = train_test_split(dtm, train['CategoryInTamil'], test_size=.20)

In [336]:
gnb = GaussianNB()
gnb.fit(train_features.toarray(), train_labels)
nb_pred_train = gnb.predict(train_features.toarray())
nb_pred_test = gnb.predict(test_features.toarray())

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
print(classification_report(test_labels, nb_pred_test))
print('Naive Bayes baseline: ' + str(multiclass_roc_auc_score(train_labels, nb_pred_train)))
print('Naive Bayes: ' + str(multiclass_roc_auc_score(test_labels, nb_pred_test)))